# **Задание 2. Работа с массивами и OpenMP**

Diana Kim ADA-2403M

**Практическое задание (25 баллов)**

Реализуйте программу на C++, которая:
1. Создаёт массив из 10 000 случайных чисел.
2. Находит минимальное и максимальное значения массива:
*   в последовательной реализации;
*   с использованием OpenMP для параллельной обработки.
3. Сравнивает время выполнения обеих реализаций и формулирует выводы.

In [10]:
%%writefile Task2.cpp

#include <iostream>         // для ввода и вывода
#include <omp.h>        // подключение OpenMP
#include <chrono>         // для измерения времени
#include <ctime>        // для time()
#include <cstdlib>          // для rand() и srand()

using namespace std;    // чтобы не писать std:: перед cout


int main() {
    const int N = 10000;          // размер массива 10000 элементов
    int* arr = new int[N];        // динамически выделяет массив
    srand((unsigned)time(0));         // инициализирует генератор случайных чисел

    for (int i = 0; i < N; i++) {          // цикл для заполнения массива
        arr[i] = rand() % 100000 + 1;          // случайные числа от 1 до 100000
    }


// ПОСЛЕДОВАТЕЛЬНЫЙ ПОИСК
    auto start_sequential = chrono::high_resolution_clock::now(); // время начала поиска

    int min_sequential = arr[0];           // считает первый элемент минимальным
    int max_sequential = arr[0];              // считает первый элемент максимальным

    for (int i = 1; i < N; i++) {            // один поток проходит по массиву
        if (arr[i] < min_sequential)           // проверяет, меньше ли текущий элемент
            min_sequential = arr[i];          // обновляет минимум
        if (arr[i] > max_sequential)          // проверяет, больше ли текущий элемент
            max_sequential = arr[i];         // обновляет максимум
    }

    auto end_sequential = chrono::high_resolution_clock::now();   // время окончания поиска
    chrono::duration<double, milli> time_sequential =
        end_sequential - start_sequential;          // длительность последовательного поиска
// ПОСЛЕДОВАТЕЛЬНЫЙ ПОИСК


// ПАРАЛЛЕЛЬНЫЙ ПОИСК
    auto start_parallel = chrono::high_resolution_clock::now();     // время начала поиска

    int min_parallel = arr[0];         // общий минимум для параллельной версии
    int max_parallel = arr[0];            // общий максимум для параллельной версии

#pragma omp parallel              // запускает параллельную область
    {
        int localMin = arr[0];           // локальный минимум потока
        int localMax = arr[0];             // локальный максимум потока

#pragma omp for nowait             // делит цикл между потоками
        for (int i = 1; i < N; i++) {            // каждый поток обрабатывает свою часть массива
            if (arr[i] < localMin)            // обновляет локальный минимум
                localMin = arr[i];
            if (arr[i] > localMax)            // обновляет локальный максимум
                localMax = arr[i];
        }

#pragma omp critical         // один поток за раз обновляет общий результат
        {   if (localMin < min_parallel) min_parallel = localMin;           // обновляет общий минимум
            if (localMax > max_parallel) max_parallel = localMax;           // обновляет общий максимум
        }
    }

    auto end_parallel = chrono::high_resolution_clock::now();        // время окончания поиска
    chrono::duration<double, milli> time_parallel =
        end_parallel - start_parallel;                  // длительность параллельного поиска
// ПАРАЛЛЕЛЬНЫЙ ПОИСК


    cout << "Array size: " << N << endl;

    cout << "\nSequential min: " << min_sequential << endl;             // выводит последовательный min
    cout << "Sequential max: " << max_sequential << endl;           // выводит последовательный max
    cout << "Sequential time: " << time_sequential.count() << " ms" << endl;      // выводит время последовательной версии

    cout << "\nParallel min: " << min_parallel << endl;       // выводит параллельный min
    cout << "Parallel max: " << max_parallel << endl;         // выводит параллельный max
    cout << "Parallel time: " << time_parallel.count() << " ms" << endl;        // выводит время параллельной версии

    cout << "\nOpenMP threads: " << omp_get_max_threads() << endl; // выводит количество потоков OpenMP

    delete[] arr;                           // освобождает память
    arr = nullptr;                          // обнуляет указатель

    return 0;                               // завершение программы
}

Overwriting Task2.cpp


In [9]:
!g++ Task2.cpp -fopenmp -O2 -o Task2
!./Task2

Array size: 10000

Sequential min: 13
Sequential max: 100000
Sequential time: 0.00799 ms

Parallel min: 13
Parallel max: 100000
Parallel time: 0.101722 ms

OpenMP threads: 2
